In [14]:
# DRIFTS stacked spectra — 2.57" x 1.82", fonts 11/10/9
# Wrapped y-label, 2000 as minor tick, legend never overlaps (measured & iterated)
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib as mpl
from matplotlib.ticker import FixedLocator

# ===== Small-panel & style =====
PANEL_W_IN, PANEL_H_IN = 2.57, 1.82
EXPORT_DPI = 600
AXIS_FS, TICK_FS, LEGEND_FS = 11, 10, 9

mpl.rcParams['axes.grid'] = False
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype']  = 42

def small_panel_ax():
    fig, ax = plt.subplots(figsize=(PANEL_W_IN, PANEL_H_IN))
    ax.tick_params(axis='both', which='major', labelsize=TICK_FS, length=3, width=0.8, direction='out')
    ax.tick_params(axis='x', which='minor', length=2, width=0.6, direction='out')
    for sp in ax.spines.values():
        sp.set_linewidth(0.8)
    return fig, ax

def ensure_ylabel_not_clipped(fig, ax, pad_px=3, min_left=0.12, max_left=0.40):
    fig.canvas.draw()
    r = fig.canvas.get_renderer()
    w = ax.yaxis.label.get_window_extent(renderer=r).width
    need = max(min_left, min(max_left, (w + pad_px) / (fig.get_figwidth()*fig.dpi)))
    fig.subplots_adjust(left=need)

def reserve_space_for_legend(fig, ax, leg, pad_px=4, floor_right=0.40):
    """
    Iteratively reduce subplot 'right' until the legend's left edge is to the
    right of the axes' right edge by at least pad_px.
    """
    for _ in range(4):
        fig.canvas.draw()
        r = fig.canvas.get_renderer()
        L = leg.get_window_extent(renderer=r)
        A = ax.get_window_extent(renderer=r)
        overhang_px = A.x1 - L.x0 + pad_px  # >0 means legend intrudes into axes
        if overhang_px > 0:
            delta = overhang_px / (fig.get_figwidth()*fig.dpi)
            new_right = max(floor_right, fig.subplotpars.right - delta)
            if abs(new_right - fig.subplotpars.right) < 1e-3:
                break
            fig.subplots_adjust(right=new_right)
        else:
            break

# ===== Inputs =====
file_paths = {
    "Fl-MSN":  r"C:\Users\Talaial Alina\OneDrive - UCB-O365\Courses\Year 1\Fall Semester Aug-Dec 2020\CHEN 5840 - Independent Study\Hydrogels\FTIR\Rhodamine Particle FTIR\23JUN25\Fl-MSN.txt",
    "Fl-HMSN": r"C:\Users\Talaial Alina\OneDrive - UCB-O365\Courses\Year 1\Fall Semester Aug-Dec 2020\CHEN 5840 - Independent Study\Hydrogels\FTIR\Rhodamine Particle FTIR\23JUN25\Fl-HMSN.txt"
}
label_map = {"Fl-MSN": "MSN", "Fl-HMSN": "DBPC HMSN"}
colors    = {"DBPC HMSN": "#D55E00", "MSN": "#009E73", "1X PBS": "#0072B2"}
baseline  = {"Fl-MSN": 3.0, "Fl-HMSN": 5.0}

# ===== Output folder = same as inputs =====
dirs = {os.path.dirname(p) for p in file_paths.values()}
out_dir = next(iter(dirs)) if len(dirs)==1 else (os.path.commonpath(list(dirs)) or next(iter(dirs)))
os.makedirs(out_dir, exist_ok=True)
OUT_PATH = os.path.join(out_dir, "FTIR_spectra_plot.png")

# ===== Load & normalize =====
data = {}
for k, path in file_paths.items():
    df = pd.read_csv(path, sep=None, engine='python', header=None, names=["Wavenumber", "Transmittance"])
    df = df[(df["Wavenumber"] <= 3500) & (df["Wavenumber"] >= 700)].copy()
    m = df["Transmittance"].max()
    if m: df["Transmittance"] /= m
    data[k] = df

# ===== Plot =====
fig, ax = small_panel_ax()
lines, labels = [], []
for k, df in data.items():
    lab = label_map[k]
    ln, = ax.plot(df["Wavenumber"], df["Transmittance"] + baseline.get(k,0.0),
                  label=lab, color=colors[lab], linewidth=3)
    lines.append(ln); labels.append(lab)

ax.invert_xaxis()
ax.set_xlabel("Wavenumber (cm⁻¹)", fontsize=AXIS_FS)
ax.set_ylabel("Transmittance\nNormalized to Peak", fontsize=AXIS_FS, labelpad=2)
ax.set_yticks([]); ax.set_yticklabels([])

# Major ticks at 3000 and 1000; 2000 as minor (unlabeled)
ax.set_xticks([3000, 1000])
ax.set_xticklabels(["3000", "1000"], fontsize=TICK_FS)
ax.xaxis.set_minor_locator(FixedLocator([2000]))

ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)

# Initial margins; we'll tighten left and reserve right precisely for legend
fig.subplots_adjust(left=0.16, right=0.86, bottom=0.22, top=0.98)
ensure_ylabel_not_clipped(fig, ax, pad_px=3, min_left=0.14, max_left=0.34)

# Legend anchored to figure's top-right (grows left); then reserve space precisely
leg = fig.legend(lines, labels, loc='upper right', bbox_to_anchor=(0.995, 0.995),
                 fontsize=LEGEND_FS, frameon=True, fancybox=True, framealpha=1.0)
leg.get_frame().set_edgecolor("#D0D0D0"); leg.get_frame().set_linewidth(0.8)
reserve_space_for_legend(fig, ax, leg, pad_px=4, floor_right=0.42)

# ===== Save (fixed canvas size; no tight bbox) =====
plt.savefig(OUT_PATH, dpi=EXPORT_DPI)
plt.close(fig)
print(f"Saved {OUT_PATH} at {PANEL_W_IN} x {PANEL_H_IN} in "
      f"({int(PANEL_W_IN*EXPORT_DPI)} x {int(PANEL_H_IN*EXPORT_DPI)} px)")


Saved C:\Users\Talaial Alina\OneDrive - UCB-O365\Courses\Year 1\Fall Semester Aug-Dec 2020\CHEN 5840 - Independent Study\Hydrogels\FTIR\Rhodamine Particle FTIR\23JUN25\FTIR_spectra_plot.png at 2.57 x 1.82 in (1542 x 1092 px)
